In [ ]:
pip install --upgrade scikit-learn imbalanced-learn


In [ ]:
# edgevit.py

from collections import OrderedDict
import torch
import torch.nn as nn
from functools import partial
import torch.nn.functional as F
import math
from timm.models.vision_transformer import _cfg
from timm.models import register_model
from timm.models.layers import trunc_normal_, DropPath, to_2tuple


class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class CMlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Conv2d(in_features, hidden_features, 1)
        self.act = act_layer()
        self.fc2 = nn.Conv2d(hidden_features, out_features, 1)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


import torch
import torch.nn as nn

class SEBlock(nn.Module):
    def __init__(self, dim, reduction=4):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim // reduction)  
        self.fc2 = nn.Linear(dim // reduction, dim)  
        self.act = nn.ReLU() 
        self.sigmoid = nn.Sigmoid() 

    def forward(self, x):
        """
        Input: x of shape [B, C, H, W]  (Batch, Channels, Height, Width)
        Output: x of shape [B, C, H, W] (Reweighted Feature Map)
        """
        B, C, H, W = x.shape

        # Squeeze - Global Average Pooling (GAP)
        w = x.mean(dim=[2, 3])  # Shape: [B, C]

        # Excite - Learn importance of channels
        w = self.fc1(w) 
        w = self.act(w)  
        w = self.fc2(w)  
        w = self.sigmoid(w) 

        #Multiply attention weights with original feature map
        w = w.unsqueeze(-1).unsqueeze(-1) 
        return x * w  


class GlobalSparseAttn(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., sr_ratio=1):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        # self.upsample = nn.Upsample(scale_factor=sr_ratio, mode='nearest')
        self.sr = sr_ratio
        if self.sr > 1:
            self.sampler = nn.AvgPool2d(1, sr_ratio)
            kernel_size = sr_ratio
            self.LocalProp = nn.ConvTranspose2d(dim, dim, kernel_size, stride=sr_ratio, groups=dim)
            self.norm = nn.LayerNorm(dim)
        else:
            self.sampler = nn.Identity()
            self.upsample = nn.Identity()
            self.norm = nn.Identity()

    def forward(self, x, H: int, W: int):
        B, N, C = x.shape
        if self.sr > 1.:
            x = x.transpose(1, 2).reshape(B, C, H, W)
            x = self.sampler(x)
            x = x.flatten(2).transpose(1, 2)

        qkv = self.qkv(x).reshape(B, -1, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, -1, C)

        if self.sr > 1:
            x = x.permute(0, 2, 1).reshape(B, C, int(H / self.sr), int(W / self.sr))
            x = self.LocalProp(x)
            x = x.reshape(B, C, -1).permute(0, 2, 1)
            x = self.norm(x)

        x = self.proj(x)
        x = self.proj_drop(x)
        return x


# class LocalAgg(nn.Module):
#     def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
#                  drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
#         super().__init__()
#         self.pos_embed = nn.Conv2d(dim, dim, 3, padding=1, groups=dim)
#         self.norm1 = nn.BatchNorm2d(dim)
#         self.conv1 = nn.Conv2d(dim, dim, 1)
#         self.conv2 = nn.Conv2d(dim, dim, 1)
#         self.attn = nn.Conv2d(dim, dim, 5, padding=2, groups=dim)
#         self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
#         self.norm2 = nn.BatchNorm2d(dim)
#         mlp_hidden_dim = int(dim * mlp_ratio)
#         self.mlp = CMlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
#
#     def forward(self, x):
#         x = x + self.pos_embed(x)
#         x = x + self.drop_path(self.conv2(self.attn(self.conv1(self.norm1(x)))))
#         x = x + self.drop_path(self.mlp(self.norm2(x)))
#         return x

class LocalAgg(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.pos_embed = nn.Conv2d(dim, dim, 3, padding=1, groups=dim)  # Positional encoding
        self.attn = nn.Conv2d(dim, dim, 5, padding=2, groups=dim)  # Local attention
        self.norm1 = nn.BatchNorm2d(dim)
        self.norm2 = nn.BatchNorm2d(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = CMlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.se = SEBlock(dim) 

    def forward(self, x):
        identity = x

       
        x = x + self.pos_embed(x)
        x = x + self.attn(self.norm1(x))

     
        x = self.se(x)

       
        x = x + self.mlp(self.norm2(x))
        return x + identity  # Residual connection



class SelfAttn(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, sr_ratio=1.):
        super().__init__()
        self.pos_embed = nn.Conv2d(dim, dim, 3, padding=1, groups=dim)
        self.norm1 = norm_layer(dim)
        self.attn = GlobalSparseAttn(
            dim,
            num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
            attn_drop=attn_drop, proj_drop=drop, sr_ratio=sr_ratio)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
        # global layer_scale
        # self.ls = layer_scale

    def forward(self, x):
        x = x + self.pos_embed(x)
        B, N, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        x = x + self.drop_path(self.attn(self.norm1(x), H, W))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        x = x.transpose(1, 2).reshape(B, N, H, W)
        return x


class LGLBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, sr_ratio=1.):
        super().__init__()

        if sr_ratio > 1:
            self.LocalAgg = LocalAgg(dim, num_heads, mlp_ratio, qkv_bias, qk_scale, drop, attn_drop, drop_path,
                                     act_layer, norm_layer)
        else:
            self.LocalAgg = nn.Identity()

        self.SelfAttn = SelfAttn(dim, num_heads, mlp_ratio, qkv_bias, qk_scale, drop, attn_drop, drop_path, act_layer,
                                 norm_layer, sr_ratio)

    def forward(self, x):
        x = self.LocalAgg(x)
        x = self.SelfAttn(x)
        return x


class PatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """

    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches
        self.norm = nn.LayerNorm(embed_dim)
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x)
        B, C, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        x = self.norm(x)
        x = x.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()
        return x


class EdgeVit(nn.Module):
    """ Vision Transformer
    A PyTorch impl of : `An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale`  -
        https://arxiv.org/abs/2010.11929
    """

    def __init__(self, depth=[1, 2, 5, 3], img_size=224, in_chans=3, num_classes=1000, embed_dim=[48, 96, 240, 384],
                 head_dim=64, mlp_ratio=4., qkv_bias=True, qk_scale=None, representation_size=None,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0., norm_layer=None, sr_ratios=[4, 2, 2, 1], **kwargs):
        """
        Args:
            depth (list): depth of each stage
            img_size (int, tuple): input image size
            in_chans (int): number of input channels
            num_classes (int): number of classes for classification head
            embed_dim (list): embedding dimension of each stage
            head_dim (int): head dimension
            mlp_ratio (int): ratio of mlp hidden dim to embedding dim
            qkv_bias (bool): enable bias for qkv if True
            qk_scale (float): override default qk scale of head_dim ** -0.5 if set
            representation_size (Optional[int]): enable and set representation layer (pre-logits) to this value if set
            drop_rate (float): dropout rate
            attn_drop_rate (float): attention dropout rate
            drop_path_rate (float): stochastic depth rate
            norm_layer (nn.Module): normalization layer
        """
        super().__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)

        self.patch_embed1 = PatchEmbed(
            img_size=img_size, patch_size=4, in_chans=in_chans, embed_dim=embed_dim[0])
        self.patch_embed2 = PatchEmbed(
            img_size=img_size // 4, patch_size=2, in_chans=embed_dim[0], embed_dim=embed_dim[1])
        self.patch_embed3 = PatchEmbed(
            img_size=img_size // 8, patch_size=2, in_chans=embed_dim[1], embed_dim=embed_dim[2])
        self.patch_embed4 = PatchEmbed(
            img_size=img_size // 16, patch_size=2, in_chans=embed_dim[2], embed_dim=embed_dim[3])

        self.pos_drop = nn.Dropout(p=drop_rate)
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depth))]  # stochastic depth decay rule
        num_heads = [dim // head_dim for dim in embed_dim]
        self.blocks1 = nn.ModuleList([
            LGLBlock(
                dim=embed_dim[0], num_heads=num_heads[0], mlp_ratio=mlp_ratio[0], qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer,
                sr_ratio=sr_ratios[0])
            for i in range(depth[0])])
        self.blocks2 = nn.ModuleList([
            LGLBlock(
                dim=embed_dim[1], num_heads=num_heads[1], mlp_ratio=mlp_ratio[1], qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i + depth[0]], norm_layer=norm_layer,
                sr_ratio=sr_ratios[1])
            for i in range(depth[1])])
        self.blocks3 = nn.ModuleList([
            LGLBlock(
                dim=embed_dim[2], num_heads=num_heads[2], mlp_ratio=mlp_ratio[2], qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i + depth[0] + depth[1]], norm_layer=norm_layer,
                sr_ratio=sr_ratios[2])
            for i in range(depth[2])])
        self.blocks4 = nn.ModuleList([
            LGLBlock(
                dim=embed_dim[3], num_heads=num_heads[3], mlp_ratio=mlp_ratio[3], qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i + depth[0] + depth[1] + depth[2]],
                norm_layer=norm_layer, sr_ratio=sr_ratios[3])
            for i in range(depth[3])])
        self.norm = nn.BatchNorm2d(embed_dim[-1])

        # Representation layer
        if representation_size:
            self.num_features = representation_size
            self.pre_logits = nn.Sequential(OrderedDict([
                ('fc', nn.Linear(embed_dim, representation_size)),
                ('act', nn.Tanh())
            ]))
        else:
            self.pre_logits = nn.Identity()

        # Classifier head
        self.head = nn.Linear(embed_dim[-1], num_classes) if num_classes > 0 else nn.Identity()

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'pos_embed', 'cls_token'}

    def get_classifier(self):
        return self.head

    def reset_classifier(self, num_classes, global_pool=''):
        self.num_classes = num_classes
        self.head = nn.Linear(self.embed_dim, num_classes) if num_classes > 0 else nn.Identity()

    def forward_features(self, x):
        x = self.patch_embed1(x)
        x = self.pos_drop(x)
        for blk in self.blocks1:
            x = blk(x)
        x = self.patch_embed2(x)
        for blk in self.blocks2:
            x = blk(x)
        x = self.patch_embed3(x)
        for blk in self.blocks3:
            x = blk(x)
        x = self.patch_embed4(x)
        for blk in self.blocks4:
            x = blk(x)
        x = self.norm(x)
        x = self.pre_logits(x)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = x.flatten(2).mean(-1)
        x = self.head(x)
        return x


@register_model
def edgevit_xxs(pretrained=True, **kwargs):
    model = EdgeVit(
        depth=[1, 1, 3, 2],
        embed_dim=[36, 72, 144, 288], head_dim=36, mlp_ratio=[4] * 4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), sr_ratios=[4, 2, 2, 1], **kwargs)
    model.default_cfg = _cfg()
    return model


@register_model
def edgevit_xs(pretrained=True, **kwargs):
    model = EdgeVit(
        depth=[1, 1, 3, 1],
        embed_dim=[48, 96, 240, 384], head_dim=48, mlp_ratio=[4] * 4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), sr_ratios=[4, 2, 2, 1], **kwargs)
    model.default_cfg = _cfg()
    return model


@register_model
def edgevit_s(pretrained=True, **kwargs):
    model = EdgeVit(
        depth=[1, 2, 5, 3],
        embed_dim=[48, 96, 240, 384], head_dim=48, mlp_ratio=[4] * 4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), sr_ratios=[4, 2, 2, 1], **kwargs)
    model.default_cfg = _cfg()
    return model
    
@register_model
def edgevit_m(pretrained=False, **kwargs):
    model = EdgeVit(
        depth=[2, 3, 6, 4],  # Increased depth for more layers per stage
        embed_dim=[64, 128, 320, 512],  # Wider embeddings for higher capacity
        head_dim=64,  # Increased head dimension for finer attention
        mlp_ratio=[4, 4, 4.5, 4.5],  # Slightly higher MLP ratio in later stages
        qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6),
        sr_ratios=[4, 2, 2, 1],  # Kept the same, as these are tied to spatial reduction
        **kwargs
    )
    model.default_cfg = _cfg()
    return model


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from collections import Counter


CK_EMOTION_MAP = {
    0: "angry",     # 0: Anger
    1: "disgust",   # 1: Disgust
    2: "fear",      # 2: Fear
    3: "happy",     # 3: Happiness
    4: "sad",       # 4: Sadness
    5: "surprise",  # 5: Surprise
    6: "neutral"    # 6: Neutral
   
}


CK_TO_FER_MAP = {
    0: 2,  # angry: 2
    1: 1,  # disgust: 1
    2: 4,  # fear: 4
    3: 6,  # happy: 6
    4: 0,  # sad: 0
    5: 5,  # surprise: 5
    6: 3,  # neutral: 3
}

class CKPlusDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform
    
    def frequency_transform(self, image):
        """Convert image into 3-channel format (Original, Low-Pass, High-Pass)."""
        if len(image.shape) == 3 and image.shape[-1] == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        image = image.astype(np.float32) / 255.0
        
        # Low-Pass Filter (Gaussian Blur)
        low_pass = cv2.GaussianBlur(image, (7, 7), 1)
        
        # High-Pass Filter (Sobel Edge Detection)
        sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
        high_pass = cv2.magnitude(sobelx, sobely)
        high_pass = high_pass / np.max(high_pass) if np.max(high_pass) != 0 else high_pass  # Normalize
        
        # Stack into 3-channel image
        freq_image = np.stack([image, low_pass, high_pass], axis=-1)
        freq_image = np.transpose(freq_image, (2, 0, 1))  # Convert HWC to CHW
        
        return freq_image.astype(np.float32)
    
    def __getitem__(self, index):
        # Rreshape to 48x48
        pixels = self.data[index]
        image = pixels.reshape(48, 48)
        
     
        image = self.frequency_transform(image)
        
        # Convert (C, H, W) -> (H, W, C) for PIL
        image = np.transpose(image, (1, 2, 0))
        
   
        image = Image.fromarray((image * 255).astype(np.uint8))
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        return image, self.labels[index]
    
    def __len__(self):
        return len(self.data)

def load_ck_plus_dataset(csv_path, test_size=0.2, random_state=42):
    """
    Load the CK+ dataset from CSV, excluding the contempt class (emotion=7)
    """

    df = pd.read_csv(csv_path)
    
    # Filter out contempt class (emotion=7)
    df = df[df['emotion'] != 7]
    
    # Convert pixel strings to numpy arrays
    pixels = df['pixels'].apply(lambda x: np.array([int(p) for p in x.split()], dtype=np.uint8))
    pixels = np.stack(pixels.values)
    
    # Mapping  to FER indices for consistency
    emotions = df['emotion'].values
    mapped_emotions = np.array([CK_TO_FER_MAP[e] for e in emotions])
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        pixels, mapped_emotions, test_size=test_size, random_state=random_state, stratify=mapped_emotions
    )
    

    print("Original Class Distribution:", Counter(mapped_emotions))
    
   
    oversampler = RandomOverSampler(random_state=random_state)
    X_train_reshaped = X_train.reshape(X_train.shape[0], -1)  # Flatten for oversampling
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_reshaped, y_train)
    X_train_resampled = X_train_resampled.reshape(-1, 48*48)  # Keep as flattened arrays
    
    print("Train Class Distribution after oversampling:", Counter(y_train_resampled))
    print("Test Class Distribution:", Counter(y_test))
    
    return X_train_resampled, y_train_resampled, X_test, y_test

def create_ck_plus_dataloaders(csv_path, batch_size=32, num_workers=4):
    """
    Create train and test DataLoaders for the CK+ dataset
    """
    # Load and split the dataset
    X_train, y_train, X_test, y_test = load_ck_plus_dataset(csv_path)
    
    # Transformation Definition
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomRotation(degrees=20),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    
  
    train_dataset = CKPlusDataset(X_train, y_train, transform=train_transform)
    test_dataset = CKPlusDataset(X_test, y_test, transform=test_transform)
    
 
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=num_workers,
        pin_memory=True
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=num_workers,
        pin_memory=True
    )
    
    return train_loader, test_loader




csv_path = "/kaggle/input/ckdataset/ckextended.csv"
    
train_loader, test_loader = create_ck_plus_dataloaders(csv_path)

for images, labels in train_loader:
        print(f"Batch shape: {images.shape}, Labels: {labels.shape}")
        break

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from PIL import Image
import cv2


import torchvision.transforms as transforms

def visualize_sample(dataset, index):
    """
    Visualize a single sample from the dataset showing the original image
    and the three channels after frequency transformation
    """

    raw_pixels = dataset.data[index]
    raw_image = raw_pixels.reshape(48, 48)
    
    # Create the frequency transformed image
    freq_image = dataset.frequency_transform(raw_image)
    
    # Get the label
    label = dataset.labels[index]
    fer_label = label  # This is already mapped to FER index
    
    # Get the emotion name
    emotion_name = None
    for ck_idx, fer_idx in CK_TO_FER_MAP.items():
        if fer_idx == fer_label:
            emotion_name = CK_EMOTION_MAP[ck_idx]
            break
    
   
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
   
    axes[0].imshow(raw_image, cmap='gray')
    axes[0].set_title(f"Original Image\nEmotion: {emotion_name} (FER idx: {fer_label})")
    axes[0].axis('off')
    
  
    channel_names = ["Original", "Low-Pass", "High-Pass"]
    for i in range(3):
        axes[i+1].imshow(freq_image[i], cmap='gray')
        axes[i+1].set_title(f"Channel {i}: {channel_names[i]}")
        axes[i+1].axis('off')
    
    plt.tight_layout()
    return fig

def visualize_random_samples(csv_path, num_samples=3):
    """
    Visualize random samples from the CK+ dataset after preprocessing
    """

    X_train, y_train, X_test, y_test = load_ck_plus_dataset(csv_path)
    
    train_dataset = CKPlusDataset(X_train, y_train, transform=None)
    
    indices = random.sample(range(len(train_dataset)), num_samples)
    
    figures = []
    for idx in indices:
        fig = visualize_sample(train_dataset, idx)
        figures.append(fig)
    
    plt.show()
    
    return figures

def visualize_batch_from_loader(train_loader):
    """
    Visualize a batch from the data loader to check tensor shapes and transforms
    """
   
    images, labels = next(iter(train_loader))
    

    print(f"Batch shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")
    
   
    for i in range(min(3, len(images))):
        # Convert tensor to numpy array (C, H, W) -> (H, W, C)
        img = images[i].numpy().transpose(1, 2, 0)
        
        # Denormalize the image: img = (img * std) + mean
        img = (img * 0.5) + 0.5
        
        # Clipping  values to [0, 1] range
        img = np.clip(img, 0, 1)
        
        label = labels[i].item()
        
        emotion_name = None
        for ck_idx, fer_idx in CK_TO_FER_MAP.items():
            if fer_idx == label:
                emotion_name = CK_EMOTION_MAP[ck_idx]
                break
        
        # Create figure
        plt.figure(figsize=(15, 5))
        
        # Display each channel separately
        for c in range(3):
            plt.subplot(1, 3, c+1)
            plt.imshow(img[:, :, c], cmap='gray')
            plt.title(f"Channel {c}")
            plt.axis('off')
        
        plt.suptitle(f"Emotion: {emotion_name} (FER idx: {label})")
        plt.tight_layout()
        plt.show()



csv_path = "/kaggle/input/ckdataset/ckextended.csv"  


train_loader, test_loader = create_ck_plus_dataloaders(csv_path, batch_size=16)

print("Visualizing random samples from the dataset...")
visualize_random_samples(csv_path, num_samples=3)


# print("\nVisualizing samples from the data loader...")
# visualize_batch_from_loader(train_loader)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from tqdm import tqdm



import torch
import torch.nn as nn

def load_pretrained_model_from_url(url, num_classes=7):
    """
    Load a pre-trained EdgeViT_XXS model from a direct URL and replace the classifier head.
    
    Args:
        url (str): Direct URL to the .pth file.
        num_classes (int): Number of output classes.
    """

    model = edgevit_xxs(num_classes=num_classes) 
    

    try:
        checkpoint = torch.hub.load_state_dict_from_url(url, map_location='cpu', progress=True)
        
   
        if 'state_dict' in checkpoint:
            checkpoint = checkpoint['state_dict']
        elif 'model_state_dict' in checkpoint:
            checkpoint = checkpoint['model_state_dict']
        
        model.load_state_dict(checkpoint, strict=False)
        print("Successfully loaded model weights from URL")
        
        return model
    
    except Exception as e:
        print(f"Error loading model from URL: {e}")
        raise


def train_model(model, dataloaders, criterion, optimizer, scheduler, device, num_epochs=25):
    """
    Train the model on the CK+ dataset
    """
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
       
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()  
            
            running_loss = 0.0
            running_corrects = 0
            

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Zero the parameter gradients
                optimizer.zero_grad()
                
                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # Backward + optimize
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # Record metrics
            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accs.append(epoch_acc.item())
            else:
                val_losses.append(epoch_loss)
                val_accs.append(epoch_acc.item())
            
            # SAve model if it's the best validation accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        print()
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')
    

    model.load_state_dict(best_model_wts)
    

    plot_training_curves(train_losses, val_losses, train_accs, val_accs)
    
    return model

def plot_training_curves(train_losses, val_losses, train_accs, val_accs):
    """
    Plot training and validation curves
    """
    plt.figure(figsize=(12, 5))
    
    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    
    # Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Training Accuracy')
    plt.plot(val_accs, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_curves.png')
    plt.show()

def evaluate_model(model, test_loader, device):
    """
    Evaluate the trained model on the test set
    """
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Print classification report
    print("\nClassification Report:")
    class_names = [CK_EMOTION_MAP[i] for i in range(7)]
    report = classification_report(all_labels, all_preds, target_names=class_names)
    print(report)
    
    # Plot confusion matrix
    plot_confusion_matrix(all_labels, all_preds, class_names)
    
    return all_preds, all_labels

def plot_confusion_matrix(y_true, y_pred, class_names):
    """
    Plot the confusion matrix
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    plt.show()



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


csv_path = "/kaggle/input/ckdataset/ckextended.csv" 


repo_id = "abhishek/edgevit_xxs_fer" 
model_name = "edgevit_xs"  


dataloaders = {
    'train': train_loader,
    'val': test_loader
}

model_url = "https://huggingface.co/abhishek/edgevit_xxs_fer/resolve/main/edgevit_distilled.pth"
model = load_pretrained_model_from_url(model_url, num_classes=7)


model = model.to(device)

criterion = nn.CrossEntropyLoss()


optimizer = optim.Adam(model.parameters(), lr=0.0001)


scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


model = train_model(model, dataloaders, criterion, optimizer, scheduler, device, num_epochs=25)


# torch.save({
#     'epoch': 25,
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(),
# }, 'edgevit_xxs_ckplus_finetuned.pth')
torch.save(model.state_dict(), "edgevit_finetuned.pth")

# Evaluate the model
evaluate_model(model, test_loader, device)

